In [1]:
from IPython.display import Math
from sage.all import *


t = var('t')
A = var('A')
B = var('B')
assume(A < 0)
assume(t < log(-1/A))

x_expr = 1 / (1 + A * exp(t))
y_expr = ((x_expr - 1) **2 / x_expr) * (B / (A ** 2) - log((1 - x_expr)  / A)) + 2 * x_expr

In [2]:
# check tha x,y solve their respective differential equations
x_sat = (diff(x_expr, t) - x_expr * (x_expr-1)).simplify_full()
y_sat = (diff(y_expr, t) - (y_expr - (x_expr + 1)) * (x_expr + 1)).simplify_full()

print(f"x satisfies: {x_sat}")
print(f"y satisfies: {y_sat}")


x satisfies: 0
y satisfies: 0


In [3]:
from sage.symbolic.integration.integral import indefinite_integral

# Compute the z solution
z = function('z')(t)

# z_deq = diff(z, t) - y_expr + x_expr + 1
# z_sol = desolve(z_deq, z, ivar=t, contrib_ode=True)
integrand = (y_expr - (x_expr + 1)).expand()
z_sols = []

def do_check_substitutions(expr):
    return expr.subs(
        log(e**t / (A * e**t + 1)) == t - log(A * e**t + 1),
    ) \
    .simplify_full()

for opnd in integrand.operands():
    z_sol_i = indefinite_integral(opnd, t)
    z_sols.append(z_sol_i)

    z_deq = (diff(z_sol_i, t) - opnd)
    z_deq = z_deq.simplify_full()
    z_deq = do_check_substitutions(z_deq)
    if z_deq != 0:
        print("Check failed!")
        print("Computing integral for:")
        display(Math(latex(opnd)))

        print("Integral:")
        display(Math(latex(z_sol_i)))

        print("Failed Check:")
        display(Math(latex(z_deq)))

z_sol_t = sum(z_sols)

print("Full Solution:")
display(Math(latex(z_sol_t)))

print("Checking:")
z_deq = (diff(z_sol_t, t) - integrand)
z_deq = z_deq.simplify_full()
z_deq = do_check_substitutions(z_deq)
display(Math(latex(z_deq)))

if z_deq != 0:
    print("Error: z solution is not correct!!!")
else:
    print("z solution is correct")


Full Solution:


<IPython.core.display.Math object>

Checking:


<IPython.core.display.Math object>

z solution is correct


In [4]:
# Next we need to substitute in x in for all the t's in z
x = var('x')

print("x(t):")
display(Math(latex(x == x_expr)))

print("z(t):")
display(Math(latex(z == z_sol_t)))

def do_x_substitutions(expr):
    return expr.expand() \
        .subs(x_expr == x) \
        .subs( (1 / x_expr).simplify_full() == 1/x) \
        .subs( ((x_expr - 1) / x_expr).simplify_full() ==((x - 1) / x)) \
        .subs( ((1 - x_expr) / x_expr).simplify_full() == (1 - x) / x) \
        .subs( (((x_expr - 1) / x_expr)*t).simplify_full()==((x - 1) / x)*t) \
        .subs( dilog((x_expr - 1) / x_expr).simplify_full() ==dilog((x - 1) / x)) \
        .subs( ((1 - x_expr) / x_expr * log(1/x)).simplify_full() ==(1 - x) / x * log(1/x)) \
        .subs( ((1-x_expr) / (A * x_expr)).simplify_full() == (1-x) / (A * x)) \
        .subs( log((1-x_expr) / (A * x_expr)).simplify_full() == log((1-x) / (A * x))) \
        .subs( log( - (x-1) / (A*x) ) == log( - (x-1) / A) + log(1/x)) \
        .subs( log(1/x) == -log(x)) \
        .simplify_full() \
        .expand()

z_sol_x = do_x_substitutions(z_sol_t)

print("z(x):")
display(Math(latex(z == z_sol_x)))

x(t):


<IPython.core.display.Math object>

z(t):


<IPython.core.display.Math object>

z(x):


<IPython.core.display.Math object>

In [17]:
# Next we need to substitute in y where it can be substituted into z
y = var('y')

print("y(t):")
display(Math(latex(y == y_expr)))

print("y(x):")
y_expr_x = y_expr.subs(
    x_expr == x,
    (1 / x_expr).simplify_full() == 1/x,
)
display(Math(latex(y == y_expr_x)))

print("z(x):")
display(Math(latex(z == z_sol_x)))

def do_y_substitutions(expr):
    return expr.expand() \
        .subs(-B / A ** 2 == -(y - 2 * x) * x / (x-1) ** 2 - log(- x / A + 1 / A)) \
        .subs(B * log(x) / A ** 2 == (y - 2 * x) * x / (x-1) ** 2 * log(x) + log(x) * log(- x / A + 1 / A)) \
        .subs(B  / (A ** 2 * x) == (y - 2 * x) * 1 / (x-1) ** 2 + 1/x * log(- x / A + 1 / A)) \
        # .simplify_full() \
        # .expand()
        # .subs( == (y - 2*x) * (1 / (x-1)) )
        # .subs( (x_expr - 1) == (x - 1)) \

z_sol_xy = do_y_substitutions(z_sol_x)

print("z(x, y):")
display(Math(latex(z == z_sol_xy)))

y(t):


<IPython.core.display.Math object>

y(x):


<IPython.core.display.Math object>

z(x):


<IPython.core.display.Math object>

z(x, y):


<IPython.core.display.Math object>

In [21]:
# So we get the final solution
C = var('C')
final_sol = y * ((x * log(x) - (x - 1)) / (x - 1) ** 2) - 2*x * ((x * log(x) - (x - 1)) / (x - 1) ** 2) + Integer(1) / Integer(2) * log(x) ** 2 + dilog((x-1) / x) + C
display(Math(latex(final_sol)))

<IPython.core.display.Math object>

In [22]:
# Check that the final solution satisfies the pde
# y - (x+1) = \partial_{y} g(x.y)( y - (x + 1))(x+1) + \partial_{x} g(x,y)x(x-1)
final_sol_x = diff(final_sol, x)
final_sol_y = diff(final_sol, y)

final_sol_pde = final_sol_y * (y - (x+1))*(x+1) + final_sol_x * x * (x-1) - (y - (x+1))
display(Math(latex(final_sol_pde.simplify_full())))



<IPython.core.display.Math object>